## Protocolos clínicos ARIA
# Escritura programática de módulos
-------
**Objetivo**: Definir funciones para escribir un protocolo clínico a partir de los datos de prescripción

Importación de módulos

In [1]:
import xml.etree.ElementTree as ET
import datetime

Definiciones de funciones

In [2]:
def parseProt(protin = 'BareBone.xml'):
    # Leer el protocolo clínico de entrada
    bbx = ET.parse(protin)
    return bbx

In [3]:
def modPreview(bbx, ID, xmlin='BareBone.xml', ApprovalStatus='Unapproved', TreatmentSite='Prostata + vvss hipofx', AssignedUsers='salud\\50724293r'):
    # Camvbiar la identificación del protocolo
    Preview = bbx.find('Preview')
    Preview.set('ID', ID)
    # Fijar el estado de aprobación
    Preview.set('ApprovalStatus', ApprovalStatus)
    # Establecer el sitio de tratamiento
    Preview.set('TreatmentSite', TreatmentSite)
    # Definir el usuario
    Preview.set('AssignedUsers', AssignedUsers)
    # Definir la fecha y hora de la creación del archivo
    creationdatetime = datetime.datetime.strftime(datetime.datetime.now(), ' %B %d %Y %H:%M:%S:%f')[:-3]
    Preview.set('LastModified', creationdatetime)
    # Formar el campo para el historial de aprobación
    ApprovalHistory = AssignedUsers + ' Created [' + creationdatetime + ' ]'
    Preview.set('ApprovalHistory', ApprovalHistory)

In [4]:
def addStructure(bbx, structureName, stColourAndStyle='Countour - Brown', searchCT=1000, vDVHLineColor=-16777216):
    # Añadir una estrctura
    StructureTemplate = bbx.find('StructureTemplate')
    Structures = StructureTemplate.find('Structures')
    Structure = ET.SubElement(Structures, 'Structure')
    Structure.set('ID', structureName)
    Structure.set('Name', structureName)
    Identification = ET.SubElement(Structure, 'Identification')
    VolumeID = ET.SubElement(Identification, 'VolumeID')
    VolumeCode = ET.SubElement(Identification, 'VolumeCode')
    VolumeType = ET.SubElement(Identification, 'VolumeType')
    VolumeType.text = 'Organ'
    VolumeCodeTable = ET.SubElement(Identification, 'VolumeCodeTable')
    StructureCode = ET.SubElement(Identification, 'StructureCode')
    TypeIndex = ET.SubElement(Structure, 'TypeIndex')
    TypeIndex.text = str(2)
    ColorAndStyle = ET.SubElement(Structure, 'ColorAndStyle')
    ColorAndStyle.text = stColourAndStyle
    SearchCTLow = ET.SubElement(Structure, 'SearchCTLow')
    SearchCTLow.text = str(searchCT)
    SearchCTHigh = ET.SubElement(Structure, 'SearchCTHigh')
    SearchCTHigh.text = str(searchCT)
    DVHLineStyle = ET.SubElement(Structure, 'DVHLineStyle')
    DVHLineStyle.text = str(0)
    DVHLineColor = ET.SubElement(Structure, 'DVHLineColor')
    DVHLineColor.text = str(vDVHLineColor)
    DVHLineWidth = ET.SubElement(Structure, 'DVHLineWidth')
    DVHLineWidth.text = str(1)
    EUDAlpha = ET.SubElement(Structure, 'EUDAlpha')
    EUDAlpha.set('xsi:nil', 'true')
    TCPAlpha = ET.SubElement(Structure, 'TCPAlpha')
    TCPAlpha.set('xsi:nil', 'true')
    TCPBeta = ET.SubElement(Structure, 'TCPBeta')
    TCPBeta.set('xsi:nil', 'true')
    TCPGamma = ET.SubElement(Structure, 'TCPGamma')
    TCPGamma.set('xsi:nil', 'true')

In [5]:
def modPhaseID(bbx, ID):
    # Modificar la identificación de la fase
    Phases = bbx.find('Phases')
    Phase = Phases.find('Phase')
    Phase.set('ID', ID)    

In [38]:
def addPlanObjetive(bbx, ID, vParameter, vDose, vTotalDose, vPrimary='false', vModifier=1):
    '''
    Function: Add a Plan Objetive
    
    Arguments:
    bbx: String
        The xml document to be modified
    ID: String
        Structure name which the plan objetive applies to
    VModifier: Int
        An index specifying the comparison operator to be used in Plan Objetive evaluation
             0: At Least (% receives more than)
             1: At Most (% receives more than). Default
             2: Minimum dose (is)
             3: Maximum dose (is)
             4: Mean dose (is)
             5: Reference point (receives)
             6: EUD (receives)
             7: Mean dose (is more than)
             8: Mean dose (is less than)
             9: Minimum dose (is more than)
            10: Maximum dose (is less than)
    vParameter: Float
        Volume percentage. Meaningless if vModifier is >= 2
    vDose: Float
        Fraction dose in Gy
    vTotalDose: Float
        Total dose in Gy
    '''
    Phases = bbx.find('Phases')
    Phase = Phases.find('Phase')
    Prescription = Phase.find('Prescription')
    Item = ET.SubElement(Prescription, 'Item')
    Item.set('ID', ID)
    Item.set('Primary', vPrimary)
    Type = ET.SubElement(Item, 'Type')
    Type.text = str(0) # hardcoded. It seems always default to zero
    Modifier = ET.SubElement(Item, 'Modifier')
    Modifier.text = str(vModifier)
    Parameter = ET.SubElement(Item, 'Parameter')
    Parameter.text = str(vParameter)
    Dose = ET.SubElement(Item, 'Dose')
    Dose.text = str(vDose)
    TotalDose = ET.SubElement(Item, 'TotalDose')
    TotalDose.text = str(vTotalDose)

In [37]:
def addQualityIndex(bbx, ID, vType, vModifier, vValue, vTypeSpecifier, vReportDQPValueInAbsoluteUnits):
    '''
    Function: Add a Quality Index
    
    Arguments:
    bbx: String
        The xml document to be modified
    ID: String
        Structure name which the quality index applies to
    vType: Int
        An index specifying the Quality Index Type
            0: ConformityIndex
            1: GradientMeasure [cm]
            2: Vxx [%/cc of volume] where xx, dose percentage, is specified by vTypeSpecifier and the %/cc of volume by vValue
            3: Vxx Gy [%/cc of volume] where xx, absolute dose in Gy, is specified by vTypeSpecifier and the %/cc of volume by vValue
            4: Dxx [%/Gy] where xx, volume percentage, is specified by vTypeSpecifier and the %/Gy of volume by vValue
            5: Dxx cc [%/Gy] where xx, absolute volume in cc, is specified by vTypeSpecifier and the %/Gy of volume by vValue
    VModifier: Int
        An index specifying the comparison operator to be used in quality index evaluation
            0: is more than
            1: is less than
            2: is
    vValue: Float
        The constrain in %, Gy o cc
    vTypeSpecifier: Float
        The dose or volume in Vdose o Dvolume in %, Gy or cc
    vReportDQPValueInAbsoluteUnits: string {'true', 'false'}
        If the constrain is specified in absolute units
    '''
    Phases = bbx.find('Phases')
    Phase = Phases.find('Phase')
    Prescription = Phase.find('Prescription')
    MeasureItem = ET.SubElement(Prescription, 'MeasureItem')
    MeasureItem.set('ID', ID)
    Type = ET.SubElement(MeasureItem, 'Type')
    Type.text = str(vType)
    Modifier = ET.SubElement(MeasureItem, 'Modifier')
    Modifier.text = str(vModifier)
    Value = ET.SubElement(MeasureItem, 'Value')
    Value.text = str(vValue)
    TypeSpecifier = ET.SubElement(MeasureItem, 'TypeSpecifier')
    TypeSpecifier.text = str(vTypeSpecifier)
    ReportDQPValueInAbsoluteUnits = ET.SubElement(MeasureItem, 'ReportDQPValueInAbsoluteUnits')
    ReportDQPValueInAbsoluteUnits.text = vReportDQPValueInAbsoluteUnits

In [7]:
def writeProt(bbx, protout = 'TestSalida.xml'):
    # Escribir el protocolo modificado
    ET.indent(bbx)
    bbx.write(protout, encoding='utf-8', xml_declaration=True)

Crear el protocolo clínico

In [48]:
bbx = parseProt()
modPreview(bbx, ID='ArmaduraTest')
modPhaseID(bbx, 'Liszt')
structureName = 'Recto'
addStructure(bbx, structureName)
addPlanObjetive(bbx, ID=structureName, vParameter=85, vDose=1, vTotalDose=20, vModifier=11)
structureName = 'PTV'
addStructure(bbx, structureName)
addPlanObjetive(bbx, structureName, vParameter=95, vDose=3*.95, vTotalDose=60*.95, vModifier=0, vPrimary='true')
addPlanObjetive(bbx, structureName, vParameter=5, vDose=3*1.07, vTotalDose=60*1.07)
structureName = 'Recto'
addQualityIndex(bbx, ID= structureName, vType=5, vModifier=1, vValue=85, vTypeSpecifier=20, vReportDQPValueInAbsoluteUnits='true')
writeProt(bbx, protout='TestSalida.xml')